In [23]:
import os
from os.path import join
import nibabel as nib
from skimage.measure import label, regionprops
import numpy as np
import math
import matplotlib.pyplot as plt
import cv2
import pandas as pd

In [24]:
def connected_component_analysis(seg, thr=0.5):
    #print(type(seg), seg.shape, seg.min(), seg.max())
    mask = seg.copy()
    mask[mask > thr] = 255
    mask[mask <= thr] = 0

    lbl = label(mask)
    blobs = regionprops(lbl)
    #raw_bbox_list = [b.bbox for b in blobs]
    #axis_major_length_list = [b.axis_major_length for b in blobs]

    return blobs

def cystregionprops(r):
    area = r.area
    major = round(r.axis_major_length, 3)
    
    try:
        minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)
    except ValueError:
        minor = np.nan

    return (area, major, minor)


In [25]:
# Folder path
seg_folder_path = '/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_arizona/cyst_segmentation'
mri_folder_path = '/data/ayc9699/dataset/pancreas_ipmn_ilkin/mayo_arizona/reoriented'
#seg_folder_path = r'C:\Users\pky0507\Desktop\ilkin\pancreas_ipmn\temporary_folder_cyst_segmentations\mayo_arizona\cyst_segmentation'
#mri_folder_path = r'C:\Users\pky0507\Desktop\ilkin\pancreas_ipmn\mayo_arizona\reoriented'

# List to store file paths
seg_file_paths = []
mri_file_paths = []
ids = []

# Iterate over files in the folder
for root, dirs, files in os.walk(seg_folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        if file.startswith(".") : continue
        seg_file_paths.append(file_path)

# Print the file paths
for path in seg_file_paths:
    id = path.split("/")[-1].split(".")[:-2][-2:]
    if id==[]: continue
    id = id[-1].split("_")[-1]
    print(id)
    mri_file_paths.append(os.path.join(mri_folder_path, f"mca_{id}.nii.gz"))


22
126
45
69
40
65
57
14
15
124
26
48
29
17
02
63
75
34
140
38


In [26]:
for i in range(len(seg_file_paths)):
    print(f"pair {i+1}")
    print(seg_file_paths[i-1], "\n", mri_file_paths[i-1])

#print(len(seg_file_paths), len(mri_file_paths))

pair 1
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_arizona/cyst_segmentation/mca_38.nii.gz 
 /data/ayc9699/dataset/pancreas_ipmn_ilkin/mayo_arizona/reoriented/mca_38.nii.gz
pair 2
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_arizona/cyst_segmentation/mca_22.nii.gz 
 /data/ayc9699/dataset/pancreas_ipmn_ilkin/mayo_arizona/reoriented/mca_22.nii.gz
pair 3
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_arizona/cyst_segmentation/mca_126.nii.gz 
 /data/ayc9699/dataset/pancreas_ipmn_ilkin/mayo_arizona/reoriented/mca_126.nii.gz
pair 4
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_arizona/cyst_segmentation/mca_45.nii.gz 
 /data/ayc9699/dataset/pancreas_ipmn_ilkin/mayo_arizona/reoriented/mca_45.nii.gz
pair 5
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_arizona/cyst_segmentation/mca_69.nii.gz 
 /data/ayc9699/data

In [27]:

def create_df():
    column_names=['Center', 'Patient', 'Pancreas_volume_ml', 'Cyst_volume_ml', 'Panc_Cyst_volume_ml', 'Diagonal_mm', 'Panc_vol_to_Diagonal', 'Num_of_Cysts']
    # Add columns using a for loop

    # Define the number of cysts
    num_cysts = 3
    for i in range(num_cysts):
        area_column_name = f'Cyst_{i+1}_vol_ml'
        major_column_name = f'Cyst_{i+1}_major_mm'
        minor_column_name = f'Cyst_{i+1}_minor_mm'
        column_names.append(area_column_name)
        column_names.append(major_column_name)
        column_names.append(minor_column_name)

    # Create an empty DataFrame
    df = pd.DataFrame(columns = column_names)

    df.head()
    return df

In [28]:

def largest3rs(r_list, voxel_sizes):

    voxel_sizes = [voxel_sizes[2], voxel_sizes[1], voxel_sizes[0]]
    sorted_r = sorted(r_list, key=lambda r: r.axis_major_length, reverse=True)
    top_3 = []
    
    for r in sorted_r[:3]:
      
        cov_matrix = r.inertia_tensor

        # Perform eigendecomposition to obtain eigenvectors and eigenvalues
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

        # Determine the dimension along which the major and minor axes lie
        major_axis_dimension = np.argmax(eigenvalues)
        minor_axis_dimension = np.argmin(eigenvalues)

        # Multiply the measurements by the corresponding voxel sizes
        axis_major_length_mm = r.major_axis_length * voxel_sizes[major_axis_dimension]
        try:
            axis_minor_length_mm = r.minor_axis_length * voxel_sizes[minor_axis_dimension]
        except ValueError:
            axis_minor_length_mm = np.nan
        
        # Use the calculated measurements and axis dimensions as needed
        #print(f"Axis Major Length (mm): {axis_major_length_mm}, Dimension: {major_axis_dimension}")
        #print(f"Axis Minor Length (mm): {axis_minor_length_mm}, Dimension: {minor_axis_dimension}")

        vol_ml = r.area * voxel_sizes[0] * voxel_sizes[1] * voxel_sizes[2]/1000

        #major = round(r.axis_major_length, 3)
        
        axis_major_length_mm = round(axis_major_length_mm, 3)
        axis_minor_length_mm = round(axis_minor_length_mm, 3)
        
        '''
        try:
            minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)
        except ValueError:
            minor = np.nan
        '''        
        
        top_3.append((vol_ml, axis_major_length_mm, axis_minor_length_mm))
        #top_3.append((vol_ml, major, minor))

    
    return top_3


In [29]:

'''
def largest3rs(r_list):
    sorted_r = sorted(r_list, key=lambda r: r.axis_major_length, reverse=True)
    top_3 = []
    
    for r in sorted_r[:3]:
        area = r.area
        major = round(r.axis_major_length, 3)
        
        try:
            minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)
        except ValueError:
            minor = np.nan
        
        top_3.append((area, major, minor))
    
    return top_3
'''

'\ndef largest3rs(r_list):\n    sorted_r = sorted(r_list, key=lambda r: r.axis_major_length, reverse=True)\n    top_3 = []\n    \n    for r in sorted_r[:3]:\n        area = r.area\n        major = round(r.axis_major_length, 3)\n        \n        try:\n            minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)\n        except ValueError:\n            minor = np.nan\n        \n        top_3.append((area, major, minor))\n    \n    return top_3\n'

In [30]:
from skimage.measure import label, regionprops
import numpy as np
def compute_diagonal_length_3D(image, voxel_size_x, voxel_size_y, voxel_size_z):
    # Ensure that image is a binary image
    #print(image.shape)
    assert np.array_equal(image, image.astype(bool)), "Input should be a binary image"

    # Label the image
    labeled_image = label(image)
    #print(np.unique(labeled_image))

    # Compute the properties of the labeled regions
    regions = regionprops(labeled_image)

    # Initialize list to store diagonal lengths
    diagonals = []
    #print("voxel",voxel_size_x, voxel_size_y, voxel_size_z)
    for region in regions:
        # Get the bounding box coordinates
        minp, minr, minc, maxp, maxr, maxc = region.bbox
        # Compute the width, height, and depth of the box
        width = (maxp - minp) * voxel_size_x
        height = (maxr - minr) * voxel_size_y
        depth = (maxc - minc) * voxel_size_z
        #print("debug--------", "depth:", (maxc - minc), "y ekseni:", (maxr - minr),  "x ekseni:", (maxp - minp))
        #print(region.bbox, width, height, depth)

        # Use Pythagorean theorem to compute the diagonal length
        diagonal = np.sqrt(width**2 + height**2 + depth**2)
        diagonals.append(diagonal)

    return diagonals

In [31]:
def get_volume_ml(label, voxel_vol_mm3):
    volume_mm3 = np.sum(label != 0) * voxel_vol_mm3
    volume_ml = volume_mm3 / 1000
    return volume_ml

In [36]:
#fig, axs = plt.subplots(1, 3, figsize=(15, 5))

df = create_df()

for eachpatientind in range(len(seg_file_paths)):
    
    pt_name = seg_file_paths[eachpatientind].split("/")[-1].split(".")[0].split("_")[-1]
    print(pt_name)
    #if seg_file_paths[eachpatientind].split("\\")[-1].split("_")[-1] == 'pass.nii': continue
    #if pt_name == '0089': continue
    # Read niftii image and segmentation
    #print(pt_name)
    img_nib = nib.load(mri_file_paths[eachpatientind])
    seg_nib = nib.load(seg_file_paths[eachpatientind])
    
    #print(pt_name, img_nib.header.get_zooms(), seg_nib.header.get_zooms())
    #print(pt_name, img_nib.shape, seg_nib.shape)

    #if pt_name in ['0140', '0159', '0172']:
        #print(seg_file_paths[eachpatientind].split("\\")[-1])
        #print(pt_name, img_nib.header.get_zooms(), seg_nib.header.get_zooms())
    #else:
    assert img_nib.header.get_zooms() == seg_nib.header.get_zooms()
    assert img_nib.shape == seg_nib.shape

    zx, zy, zz = seg_nib.header.get_zooms()
    voxel_sizes = seg_nib.header.get_zooms()
    print(voxel_sizes)

    img = img_nib.get_fdata()
    seg = seg_nib.get_fdata()
    print(np.unique(seg))

    #print('img', img.shape)
    print('seg', seg.shape)


    cysts = (seg == 5)
    pancreas = (seg == 1)

    labels = sorted(np.unique(seg))
    print("ll",labels)
    if len(labels)==2:
        panc_label = labels[1]
        pancreas = (seg == panc_label)
        cysts = (seg == 9999)

    cysts_b = cysts.astype(int)
    pancreas_b = pancreas.astype(int)
    pancreasandcyst = np.concatenate((cysts_b, pancreas_b))

    # Find Bboxes
    blobs = connected_component_analysis(cysts, thr=0.5)
    num_of_cysts = len(blobs)

    # Calculate the voxel volume (assuming isotropic voxel size)
    voxel_vol_mm3 = voxel_sizes[0] *voxel_sizes[1] * voxel_sizes[2]  # Specify the voxel volume in your desired units

    # Calculate the volume of the segmentation mask
    cysts_volume_ml = get_volume_ml(cysts, voxel_vol_mm3)
    pancreas_volume_ml = get_volume_ml(pancreas, voxel_vol_mm3)

    diagonal_lenght = max(compute_diagonal_length_3D(pancreasandcyst, voxel_sizes[0], voxel_sizes[1], voxel_sizes[2]))
    pancreas_volume_to_diagonal = pancreas_volume_ml / diagonal_lenght

    column_data = ['Mayo Arizona', pt_name, pancreas_volume_ml, cysts_volume_ml, pancreas_volume_ml + cysts_volume_ml, diagonal_lenght, pancreas_volume_to_diagonal, num_of_cysts]

    result = largest3rs(blobs, voxel_sizes)
    # Print the result
    for i, props in enumerate(result, start=1):
        #print(f"Top {i}: Area={props[0]}, Major={props[1]}, Minor={props[2]}")
        area, major, minor = props[0], props[1], props[2]
        column_data.append(area)
        column_data.append(major)
        column_data.append(minor)

    column_data.extend([math.nan] * (17 - len(column_data)))
    #print(column_data)

    df.loc[len(df)] = column_data

df

#plt.show()

22
(0.8203, 0.8203, 7.0)
[0. 1. 5.]
seg (512, 512, 30)
ll [0.0, 1.0, 5.0]
126
(0.7813, 0.7813, 7.0)
[0. 1. 5.]
seg (512, 512, 40)
ll [0.0, 1.0, 5.0]
45
(0.7227, 0.7227, 7.0)
[0. 1. 5.]
seg (512, 512, 39)
ll [0.0, 1.0, 5.0]
69
(0.8203, 0.8203, 4.0)
[0. 1. 5.]
seg (512, 512, 38)
ll [0.0, 1.0, 5.0]
40
(0.5313, 0.5313, 5.0)
[0. 1.]
seg (604, 640, 36)
ll [0.0, 1.0]
65
(0.7031, 0.7031, 7.0)
[0. 1. 5.]
seg (512, 512, 35)
ll [0.0, 1.0, 5.0]
57
(0.7813, 0.7813, 7.0)
[0. 1. 5.]
seg (512, 512, 39)
ll [0.0, 1.0, 5.0]
14
(0.989583, 0.989583, 5.0)
[0. 1. 5.]
seg (384, 324, 50)
ll [0.0, 1.0, 5.0]
15
(0.78125, 0.78125, 6.0)
[0. 1. 5.]
seg (512, 512, 56)
ll [0.0, 1.0, 5.0]
124
(0.7031, 0.7031, 9.0)
[0. 1.]
seg (512, 512, 33)
ll [0.0, 1.0]
26
(0.7813, 0.7813, 7.0)
[0. 1. 5.]
seg (512, 512, 30)
ll [0.0, 1.0, 5.0]
48
(0.7813, 0.7813, 7.0)
[0. 1. 5.]
seg (512, 512, 40)
ll [0.0, 1.0, 5.0]
29
(0.7813, 0.7813, 7.0)
[0. 1. 5.]
seg (512, 512, 40)
ll [0.0, 1.0, 5.0]
17
(0.7813, 0.7813, 7.0)
[0. 1. 5.]
seg (512, 

,Center,Patient,Pancreas_volume_ml,Cyst_volume_ml,Panc_Cyst_volume_ml,Diagonal_mm,Panc_vol_to_Diagonal,Num_of_Cysts,Cyst_1_vol_ml,Cyst_1_major_mm,Cyst_1_minor_mm,Cyst_2_vol_ml,Cyst_2_major_mm,Cyst_2_minor_mm,Cyst_3_vol_ml,Cyst_3_major_mm,Cyst_3_minor_mm
0,Mayo Arizona,22,53.305833,16.156138,69.461971,159.014427,0.335226,16,10.136446,124.363,20.038,2.868539,35.172,31.508,2.265628,27.185,23.643
1,Mayo Arizona,126,23.894660,8.665660,32.560320,105.386612,0.226733,3,8.460556,42.678,41.161,0.102552,4.837,0.000,0.102552,4.837,0.000
2,Mayo Arizona,45,53.630849,9.699546,63.330395,165.589269,0.323879,9,7.356007,28.649,35.085,2.072990,23.875,20.014,0.095058,7.821,NaN
3,Mayo Arizona,69,52.337542,7.740950,60.078492,159.085365,0.328990,8,7.006152,78.338,25.387,0.306839,32.398,5.451,0.296073,28.986,7.514
4,Mayo Arizona,40,24.727702,0.000000,24.727702,97.285467,0.254177,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Mayo Arizona,65,22.849335,3.512354,26.361689,132.936720,0.171881,19,1.965534,49.891,13.364,0.294138,15.809,11.656,0.290678,14.438,0.000
6,Mayo Arizona,57,15.946865,18.275655,34.222520,153.729126,0.103734,4,15.793037,117.724,25.564,2.401430,29.801,18.207,0.064095,4.941,0.000
7,Mayo Arizona,14,20.765517,4.421425,25.186942,157.399276,0.131929,11,3.070026,68.593,13.562,0.141995,30.532,NaN,0.328057,19.389,0.000
8,Mayo Arizona,15,69.466553,63.398438,132.864990,175.998937,0.394699,14,5.137939,123.770,13.722,57.747803,62.898,69.046,0.164795,7.347,0.000
9,Mayo Arizona,124,40.602913,0.000000,40.602913,128.637124,0.315639,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Save the DataFrame to a CSV file
df.to_csv('server_cyst_segmentation_mayo_arizona.csv', index=False)